In [1]:
from typing import Iterable
import gdspy
import numpy as np

import ruamel.yaml as yaml
import os

import qnldraw as qd
import qnldraw.junction as qj

from copy import deepcopy
from itertools import product
from pathlib import Path

from qnldraw import shapes, paths, components, Angle, Params
from qnldraw.shapes import Rectangle
from qnldraw.paths import Trace

In [13]:
def make_snail_array(idx, params, simulate_evap=True):
    ### Make a deep copy of the parameter dict ###
    params = Params(params)
    # This will help us manage layers
    chip = qd.Chip()

    ### =============== Layout Parameters =============== ###
    pad_spacing = params['pad_spacing']
    lead_vertical_separation = np.sqrt(params['squid_loop_area'])
    lead_horizontal_separation = np.sqrt(params['squid_loop_area'])

    spacing = 2*np.sqrt(params['squid_loop_area'])

    hdlayer = params['layers.highdose']
    ldlayer = params['layers.lowdose']

    ### =============== Junction Leads and Bandages =============== ###
    lead_params = params['leads']
    lead_params.update({
        'total_length': pad_spacing/2 + lead_params['pad_overlap'] - lead_horizontal_separation/2,
        'extension': lead_params['inner.width']/2 - params['wire.width']/2,
    })

    leads = qj.JunctionLead(**lead_params)
    bandage = qj.Bandage(**params['bandages'])

    lx = lead_horizontal_separation
    ly = (params['wire']['width'] + lead_vertical_separation)/2

    left_lead = leads.place((-lx / 2, 0), node='inner_lead')
    right_lead = leads.place((lx / 2, 0), node='inner_lead', mirror='y')

    left_bandage = bandage.attach(left_lead, 'origin', 'right', offset=(params['bandages.lead_overlap'], 0))
    right_bandage = bandage.attach(right_lead, 'origin', 'left', offset=(-params['bandages.lead_overlap'], 0))

    chip.add_component([left_bandage, right_bandage], 'bandages', layers=params['bandages.layer'])
    chip.add_component([left_lead, right_lead], 'leads', layers=hdlayer)

    ### =============== DOLAN JUNCTIONS =============== ###

    junction = SNAILArray(**params['junction'], wire=params['wire'])

    chip.add_component(junction.place(), 'junction', layers=[hdlayer, ldlayer])

    wires = [
        paths.Trace.between(left_lead.node('lead_0'), junction.node('lead_0'), params['wire.width']),
        paths.Trace.between(right_lead.node('lead_0'), junction.node('lead_1'), params['wire.width']),
    ]

    chip.add_component(wires, 'wires', layers=hdlayer)

    ### =============== LOW DOSE AROUND LEADS =============== ###
    lead_lowdose = qd.offset(
        [left_lead, right_lead],
        params['leads.undercut'],
        join_first=True,
        join='round',
        tolerance=1
    )

    lead_lowdose = qd.boolean(
        lead_lowdose,
        wires + [left_lead, right_lead],
        'not',
        layer=ldlayer
    )

    chip.add_component(lead_lowdose, cid='lowdose', layers=ldlayer)

    ### =============== SIMULATE EVAPORATION =============== ###
    if simulate_evap:

        cell = chip.render('junction_mask', include_refs=False)[0]

        all_polygons = cell.get_polygons(by_spec=True)

        highdose = gdspy.PolygonSet(all_polygons[(1, 0)])
        lowdose = gdspy.PolygonSet(all_polygons[(2, 0)])

        evaporated = qj.simulate_evaporation(
            lowdose, highdose, **params['evaporation']
        )

        for i, (layer, evap) in enumerate(zip(params['evaporation.layers'], evaporated)):
            chip.add_component(evap, f'evap_{i}', layers=layer)

    return chip.render(str(idx), include_refs=False)

def make_dolan(idx, params, simulate_evap=True):
    ### Make a deep copy of the parameter dict ###
    params = Params(params)
    # This will help us manage layers
    chip = qd.Chip()

    ### =============== Layout Parameters =============== ###
    pad_spacing = params['pad_spacing']
    lead_vertical_separation = np.sqrt(params['squid_loop_area'])
    lead_horizontal_separation = np.sqrt(params['squid_loop_area'])

    ur = 0.5 * np.array((lead_horizontal_separation, lead_vertical_separation))
    # Offset so inner edge lines up with SQUID loop area
    ur += 0.5 * params['wire.width']

    hdlayer = params['layers.highdose']
    ldlayer = params['layers.lowdose']

    ### =============== Junction Leads and Bandages =============== ###
    lead_params = params['leads']
    lead_params.update({
        'total_length': pad_spacing/2 + lead_params['pad_overlap'] - lead_horizontal_separation/2,
        'extension': lead_params['inner.width']/2 - params['wire.width']/2,
    })

    leads = qj.JunctionLead(**lead_params)
    bandage = qj.Bandage(**params['bandages'])

    lx = pad_spacing/2 + lead_params['pad_overlap']
    ly = (params['wire']['width'] + lead_vertical_separation)/2

    left_lead = leads.place((-lx, 0))
    right_lead = leads.place((lx, 0), mirror='y')

    left_bandage = bandage.attach(left_lead, 'origin', 'right', offset=(params['bandages.lead_overlap'], 0))
    right_bandage = bandage.attach(right_lead, 'origin', 'left', offset=(-params['bandages.lead_overlap'], 0))

    chip.add_component([left_bandage, right_bandage], 'bandages', layers=params['bandages.layer'])
    chip.add_component([left_lead, right_lead], 'leads', layers=hdlayer)

    ### =============== MANHATTAN JUNCTIONS =============== ###

    jparams = params['junction']

    x, y = left_lead.node('lead_0')
    w1 = paths.Trace(params['wire.width'], start=(x, y))
    w1_undercut = paths.CPW(params['wire.width'], gap=params['wire.undercut'], start=(x,y))
    w1.segment(-jparams['wire.width']/2 - x)
    w1_undercut.segment(-jparams['wire.width']/2 - x, layer=1)

    x, y = right_lead.node('lead_0')
    w2 = paths.Trace(params['wire.width'], start=(x, y), direction='-x')
    w2_undercut = paths.CPW(params['wire.width'], gap=params['wire.undercut'], start=(x,y), direction='-x')
    w2.segment(x - jparams['wire.width']/2)
    w2_undercut.segment(x -jparams['wire.width']/2, layer=1)

    wires = [w1, w2]

    junction = DolanJunction(**jparams).place()

    wires = [
        paths.Trace.between(
            left_lead.node('lead_0'),
            junction.node('lead0'),
            jparams['wire.width']
        ),
        paths.Trace.between(
            right_lead.node('lead_0'),
            junction.node('lead1'),
            jparams['wire.width']
        )
    ]

    chip.add_component(junction, cid='dolan', layers=[hdlayer, ldlayer])

    ### =============== LOW DOSE AROUND LEADS =============== ###
    lead_lowdose = qd.offset(
        [left_lead, right_lead],
        params['leads.undercut'],
        join_first=True,
        join='round',
        tolerance=1
    )

    lead_lowdose = qd.boolean(
        lead_lowdose,
        [left_lead, right_lead] + wires,
        'not',
        layer=2
    )

    chip.add_component(lead_lowdose, cid='lowdose', layers=ldlayer)
    chip.add_component(wires, cid='wires', layers=[hdlayer, ldlayer])

    ### =============== SIMULATE EVAPORATION =============== ###
    if simulate_evap:

        cell = chip.render('junction_mask', include_refs=False)[0]

        all_polygons = cell.get_polygons(by_spec=True)

        highdose = gdspy.PolygonSet(all_polygons[(1, 0)])
        lowdose = gdspy.PolygonSet(all_polygons[(2, 0)])

        evaporated = qj.simulate_evaporation(
            lowdose, highdose, **params['evaporation']
        )

        for i, (layer, evap) in enumerate(zip(params['evaporation.layers'], evaporated)):
            chip.add_component(evap, f'evap_{i}', layers=layer)

    return chip.render(str(idx), include_refs=False)

def make_ldolan(idx, params, simulate_evap=True):
    ### Make a deep copy of the parameter dict ###
    params = Params(params)
    # This will help us manage layers
    chip = qd.Chip()

    ### =============== Layout Parameters =============== ###
    pad_spacing = params['pad_spacing']
    lead_vertical_separation = np.sqrt(params['squid_loop_area'])
    lead_horizontal_separation = np.sqrt(params['squid_loop_area'])

    ur = 0.5 * np.array((lead_horizontal_separation, lead_vertical_separation))
    # Offset so inner edge lines up with SQUID loop area
    ur += 0.5 * params['wire.width']

    hdlayer = params['layers.highdose']
    ldlayer = params['layers.lowdose']

    ### =============== Junction Leads and Bandages =============== ###
    lead_params = params['leads']
    lead_params.update({
        'total_length': pad_spacing/2 + lead_params['pad_overlap'] - lead_horizontal_separation/2,
        'extension': lead_params['inner.width']/2 - params['wire.width']/2,
    })

    leads = qj.JunctionLead(**lead_params)
    bandage = qj.Bandage(**params['bandages'])

    lx = pad_spacing/2 + lead_params['pad_overlap']
    ly = (params['wire']['width'] + lead_vertical_separation)/2

    left_lead = leads.place((-lx, params['junction.offset'] / 2))
    right_lead = leads.place((lx, -params['junction.offset'] / 2), mirror='y')

    left_bandage = bandage.attach(left_lead, 'origin', 'right', offset=(params['bandages.lead_overlap'], -0.7))
    right_bandage = bandage.attach(right_lead, 'origin', 'left', offset=(-params['bandages.lead_overlap'], 0))

    chip.add_component([left_bandage, right_bandage], 'bandages', layers=params['bandages.layer'])
    chip.add_component([left_lead, right_lead], 'leads', layers=params['layers.lead_highdose'])

    ### =============== MANHATTAN JUNCTIONS =============== ###

    jparams = params['junction']

    x, y = left_lead.node('lead_0')
    w1 = paths.Trace(params['wire.width'], start=(x, y))
    w1_undercut = paths.CPW(params['wire.width'], gap=params['wire.undercut'], start=(x,y))
    w1.segment(-jparams['wire.width']/2 - x)
    w1_undercut.segment(-jparams['wire.width']/2 - x, layer=1)

    x, y = right_lead.node('lead_0')
    w2 = paths.Trace(params['wire.width'], start=(x, y), direction='-x')
    # w2_undercut = paths.CPW(params['wire.width'], gap=params['wire.undercut'], start=(x,y), direction='-x')
    w2.segment(x - jparams['wire.width']/2)
    # w2_undercut.segment(x -jparams['wire.width']/2, layer=1)

    wires = [w1, w2]

    junction = LJunction(**jparams).place()

    wires = [
        paths.Trace.between(
            left_lead.node('lead_0'),
            junction.node('lead0'),
            jparams['wire.width']
        ),
        paths.Trace.between(
            right_lead.node('lead_0'),
            junction.node('lead1'),
            jparams['wire.width']
        )
    ]

    chip.add_component(junction, cid='dolan', layers=[hdlayer, ldlayer])

    ### =============== LOW DOSE AROUND LEADS =============== ###
    lead_lowdose = qd.offset(
        [left_lead, right_lead],
        params['leads.undercut'],
        join_first=True,
        join='round',
        tolerance=1
    )

    lead_lowdose = qd.boolean(
        lead_lowdose,
        [left_lead, right_lead] + wires,
        'not',
        layer=2
    )

    chip.add_component(lead_lowdose, cid='lowdose', layers=params['layers.lead_lowdose'])
    chip.add_component(wires, cid='wires', layers=[hdlayer, ldlayer])

    ### =============== SIMULATE EVAPORATION =============== ###
    if simulate_evap:

        cell = chip.render('junction_mask', include_refs=False)[0]

        all_polygons = cell.get_polygons(by_spec=True)

        # print(all_polygons)

        highdose = gdspy.PolygonSet(all_polygons[(hdlayer, 0)])
        lowdose = gdspy.PolygonSet(all_polygons[(ldlayer, 0)])

        evaporated = qj.simulate_evaporation(
            lowdose, highdose, **params['evaporation']
        )

        for i, (layer, evap) in enumerate(zip(params['evaporation.layers'], evaporated)):
            chip.add_component(evap, f'evap_{i}', layers=layer)

    return chip.render(str(idx), include_refs=False)

def make_snail(idx, params, simulate_evap=True):
    ### Make a deep copy of the parameter dict ###
    params = Params(params)
    # This will help us manage layers
    chip = qd.Chip()

    ### =============== Layout Parameters =============== ###
    pad_spacing = params['pad_spacing']
    # lead_vertical_separation = np.sqrt(params['squid_loop_area'])
    # lead_horizontal_separation = np.sqrt(params['squid_loop_area'])

    # ur = 0.5 * np.array((lead_horizontal_separation, lead_vertical_separation))
    # Offset so inner edge lines up with SQUID loop area
    # ur += 0.5 * params['wire.width']

    hdlayer = params['layers.highdose']
    ldlayer = params['layers.lowdose']

    ### =============== Junction Leads and Bandages =============== ###
    lead_params = params['leads']
    lead_params.update({
        # 'total_length': pad_spacing/2 + lead_params['pad_overlap'] - lead_horizontal_separation/2,
        'extension': lead_params['inner.width']/2 - params['wire.width']/2,
    })

    leads = qj.JunctionLead(**lead_params)
    bandage = qj.Bandage(**params['bandages'])

    lx = pad_spacing/2 + lead_params['pad_overlap']
    # ly = (params['wire']['width'] + lead_vertical_separation)/2

    left_lead = leads.place((-lx, 0))
    right_lead = leads.place((lx, 0), mirror='y')

    left_bandage = bandage.attach(left_lead, 'origin', 'right', offset=(params['bandages.lead_overlap'], 0))
    right_bandage = bandage.attach(right_lead, 'origin', 'left', offset=(-params['bandages.lead_overlap'], 0))

    chip.add_component([left_bandage, right_bandage], 'bandages', layers=params['bandages.layer'])
    chip.add_component([left_lead, right_lead], 'leads', layers=hdlayer)

    ### =============== MANHATTAN JUNCTIONS =============== ###

    jparams = params['junction']
    aparams = params['array']
    aparams['wire'] = params['junction.wire']
    
    xoffset, yoffset = params['loop'].get('offset', (0, 0))
    
    snl = SNAIL(jparams, aparams, params['loop']).place((xoffset, -yoffset))
    
    snl_copy = SNAIL(jparams, aparams, params['loop']).place((0,0))
    chip.add_component(snl, cid='snail', layers=[hdlayer, ldlayer, 1, 2])
    snl_displaced = snl_copy
    if yoffset ==0:
        x, y = left_lead.node('lead_0')
        w1 = Trace.between((x, y), (snl.node('lead0')[0]+0.7,snl.node('lead0')[1]), params['wire.width'])
        x, y = right_lead.node('lead_0')
        w2 = Trace.between((x, y), (snl.node('lead1')[0],snl.node('lead1')[1]), params['wire.width'])
        wires = [w1, w2]
    else:
        x, y = left_lead.node('lead_0')
#         if lead_params['radius'] != 0:
#             w1 = Trace.between((x, y), (snl.node('lead0')[0] - lead_params['radius'] + params['wire.width']/2,snl.node('lead0')[1]), params['wire.width'])\
#                       .turn(lead_params['radius'], -90 * abs(yoffset+1e-15) / (yoffset+1e-15))
#             w1 = w1.segment(abs(snl_displaced.node('lead0')[1] - params['loop.size'][1] + 2.15 - w1.current_position()[1]) - lead_params['radius'] * 2)
#             x, y = right_lead.node('lead_0')
#             w2 = Trace.between((x, y), (snl.node('lead1')[0] + lead_params['radius'] - params['wire.width']/2,snl.node('lead1')[1]), params['wire.width'])\
#                       .turn(lead_params['radius'], +90 * abs(yoffset+1e-15) / (yoffset+1e-15))
#             w2 = w2.segment(abs(snl_displaced.node('lead1')[1] - params['loop.size'][1] + 2.85 - w2.current_position()[1]) - lead_params['radius'] * 2)
#             wires = [w1, w2]

        if True:
            print('here')
            w1 = Trace.between((x, y), (snl.node('lead0')[0]+0.7 + params['wire.width'] ,snl.node('lead0')[1]), params['wire.width'])
                      # .turn(lead_params['radius'], -90 * abs(yoffset+1e-15) / (yoffset+1e-15))
            w1_2 = Trace(width = 1, start = (w1.current_position()[0] - params['wire.width']/2, w1.current_position()[1] - params['wire.width']/2)).segment(abs(snl_displaced.node('lead0')[1] - params['loop.size'][1]/2 + 2.15 - 3 - w1.current_position()[1]), direction = '-y')
            x, y = right_lead.node('lead_0')
            w2 = Trace.between((x, y), (snl.node('lead1')[0] - params['wire.width'],snl.node('lead1')[1]), params['wire.width'])
                      # .turn(lead_params['radius'], +90 * abs(yoffset+1e-15) / (yoffset+1e-15))
            w2_2 = Trace(width = 1, start = (w2.current_position()[0] + params['wire.width']/2, w2.current_position()[1] - params['wire.width']/2)).segment(abs(snl_displaced.node('lead1')[1] - params['loop.size'][1]/2 + 2.85 - 3 - w2.current_position()[1]), direction = '-y')
            # w2 = w2.segment(abs(snl_displaced.node('lead1')[1] - params['loop.size'][1] + 2.85 - w2.current_position()[1]))
            wires = [w1, w1_2, w2, w2_2]


    # w1_undercut = paths.CPW(params['wire.width'], gap=params['wire.undercut'], start=(x,y))
    # w1.segment(-jparams['wire.width']/2 - x)
    # w1_undercut.segment(-jparams['wire.width']/2 - x, layer=1)

    # x, y = right_lead.node('lead_0')
    # w2 = paths.Trace(params['wire.width'], start=(x, y), direction='-x')
    # w2_undercut = paths.CPW(params['wire.width'], gap=params['wire.undercut'], start=(x,y), direction='-x')
    # w2.segment(x - jparams['wire.width']/2)
    # w2_undercut.segment(x -jparams['wire.width']/2, layer=1)

    

    ### =============== LOW DOSE AROUND LEADS =============== ###
    lead_lowdose = qd.offset(
        [left_lead, right_lead],
        params['leads.undercut'],
        join_first=True,
        join='round',
        tolerance=1
    )

    lead_lowdose = qd.boolean(
        lead_lowdose,
        [left_lead, right_lead] + wires,
        'not',
        layer=2
    )

    chip.add_component(lead_lowdose, cid='lowdose', layers=ldlayer)
    chip.add_component(wires, cid='wires', layers=[hdlayer, ldlayer])

    ### =============== SIMULATE EVAPORATION =============== ###
    if simulate_evap:

        cell = chip.render('junction_mask', include_refs=False)[0]

        all_polygons = cell.get_polygons(by_spec=True)

        highdose = gdspy.PolygonSet(all_polygons.get((hdlayer, 0), []) + all_polygons.get((3, 0), []))
        lowdose = gdspy.PolygonSet(all_polygons.get((ldlayer, 0), []) + all_polygons.get((4, 0), []))

        evaporated = qj.simulate_evaporation(
            lowdose, highdose, **params['evaporation']
        )

        for i, (layer, evap) in enumerate(zip(params['evaporation.layers'], evaporated)):
            chip.add_component(evap, f'evap_{i}', layers=layer)

    return chip.render(str(idx), include_refs=False)

class BridgeTrace(paths.Trace):
    def shift(self, length, shift, side, flip=True, **kwargs):
        defaults = {
            'layer': qd.qdParams['layer'],
            'datatype': qd.qdParams['datatype']
        }

        defaults.update(**kwargs)
        direction = Angle(self.direction, relative=False, unit='rad')

        width = 2*self.w + shift
        side = 1 if side == 'left' else -1

        lengths = length if isinstance(length, Iterable) else [length]
        N = len(lengths)

        polygons = []
        x = 0
        for l in lengths:
            points = np.array([
                [x, 0.5*width],
                [x, -0.5*width],
                [x + l, -0.5*width],
                [x + l, 0.5*width]
            ]) + 0.5 * side * np.array((0, shift))

            x += l
            polygons += [points]

        new_pos = np.array([0 if flip else sum(lengths), side*shift])

        ## Rotate by current direction
        for i, points in enumerate(polygons):
            polygons[i] = points * direction.cos() \
                + points[:,::-1] * direction.sin() * np.array((-1.0, 1.0))
        new_pos = new_pos * direction.cos() \
                + new_pos[::-1] * direction.sin() * np.array((-1.0, 1.0))

        ## Translate by current x, y
        for i, points in enumerate(polygons):
            polygons[i] = points + self.current_position()
        new_pos = new_pos + self.current_position()

        self.polygons.extend(polygons)
        if isinstance(defaults['layer'], Iterable):
            self.layers.extend([l for l in defaults['layer']])
        else:
            self.layers.extend([defaults['layer']]*N)
        if isinstance(defaults['datatype'], Iterable):
            self.datatypes.extend([d for d in defaults['datatype']])
        else:
            self.datatypes.extend([defaults['datatype']]*N)

        self.x, self.y = new_pos

        self.direction = (direction + (180 if flip else 0)).evaluate('rad', exact=True)

        return self

class SNAILArray(components.Component):
    __draw__ = True

    def draw(self,
        width,
        bridge,
        taper,
        wire,
        bridgegap=0,
        num=100, #number of arrays
        short=False
    ):
        bl = 0 if short else 1

        junction = BridgeTrace(width=wire['width'], direction='+y')

        junction.taper(taper, width)

        for _ in range((num + 1) // 2):
            junction.segment(bridgegap)
            junction.segment(bridge, layer=bl)

        junction.segment(bridgegap)
        junction.taper(taper, wire['width'])

        x, y = junction.current_position()
        junction.translate(0, -y/2)

        self.add(junction)
        return {'lead_0': np.array((0, -y/2)), 'lead_1': np.array((0, y/2))}
        
# class BridgeArray(components.Component):
#     __draw__ = True

#     def draw(self,
#         num,
#         width,
#         bridge,
#         gap,
#         wire,
#         shift,
#         num_segments=3,
#         bridgegap=0,
#         short=False):
#         n = round(num / (4*num_segments))

#         bl = 0 if short else 1
#         undercut = wire['undercut']

#         junction = BridgeTrace(width=width, direction='-y')
#         for seg in range(num_segments):
#             for i in range(n):
#                 junction.segment(bridgegap)
#                 junction.segment(bridge, layer=bl)

#             junction.shift(
#                 [bridgegap, undercut],
#                 shift,
#                 'left',
#                 layer=[0, 1])

#             for i in range(2*n - 1):
#                 junction.segment(bridge, layer=bl)
#                 junction.segment(bridgegap)

#             junction.segment(bridge, layer=bl)

#             junction.shift(
#                 [bridgegap, undercut],
#                 shift,
#                 'right',
#                 layer=[0, 1])

#             for i in range(n - 1):
#                 junction.segment(bridge, layer=bl)
#                 junction.segment(bridgegap)

#             if seg < num_segments - 1:
#                 junction.segment(bridge, layer=bl)

#         x, y = junction.current_position()
#         junction.translate(-x/2, -bridge/2)

#         lead1 = paths.Trace(width=width, start=(-x/2, -bridge/2), direction='+y')
#         lead2 = paths.Trace(width=width, start=junction.current_position(), direction='-y')
        
#         lead1.segment(0.5*(bridge + wire['width'])).segment(undercut, layer=1)
#         lead2.segment(0.5*(bridge + wire['width'])).segment(undercut, layer=1)

#         self.add(lead1)
#         self.add(lead2)

#         self.add(junction)
#         return {'lead_0': np.array((-(x + width)/2, 0)), 'lead_1': np.array(((x + width)/2, 0))}

# # class DolanJunction(components.Component):
# #     __draw__ = True
    
# #     def draw(self, wire, taper, finger, base, bridge=0.14):
# #         wlen1, wlen2 = wire['lengths']
# #         w1 = paths.Trace(wire['width'])
# #         w1.segment(wlen1)
# #         w1.taper(taper, finger['width'])
# #         w1.segment(finger['length'])

# #         x, y = w1.current_position()
# #         w2 = paths.Trace(base, start=(x, y))
# #         w2.segment(bridge, layer=1)
# #         w2.segment(finger['length'])
# #         w2.taper(taper, wire['width'])
# #         w2.segment(wlen2)

# #         w1.translate(-x, 0)
# #         w2.translate(-x, 0)

# #         self.add([w1, w2])

# #         return {'lead0': (-x, 0), 'lead1': w2.current_position()}

class LJunction(components.Component):
    __draw__ = True
    
    def draw(self, wire, overlap, finger, bridge, base, offset, taper):
        nodes = {}

        start = (-(taper + finger), offset/2)
        nodes['lead0'] = start
        w1 = paths.Trace(wire['width'], start=start)
        w1.taper(taper, overlap['y'])
        w1.segment(finger)
        
        print(base['extra'])
        print(offset)
        print(overlap['y'])
        print(base['width'])
        h = base['extra'] + offset + (overlap['y'] + base['width']) / 2
        junction = paths.Trace(h, start=(0, (base['extra'] + 0.5 * (overlap['y'] - base['width'])) / 2))
        print(h)
        junction.segment(bridge, layer=1)
        junction.segment(overlap['x'])
        
        start = (bridge + overlap['x'], -offset/2)
        w2 = paths.Trace(base['width'], start=start)
        w2.segment(finger)
        w2.taper(taper, wire['width'])

        self.add([w1.rotate(-np.pi/2), junction.rotate(-np.pi/2), w2.rotate(-np.pi/2)])
        nodes['lead1'] = w2.current_position()
        nodes['lead0'] = w1.current_position()
        nodes['lead0'][1] = nodes['lead0'][1]-finger-taper+0.05# added for light
        return nodes

class SNAIL(components.Component):
    __draw__ = True

    def draw(self, dolan, array, loop, num=1, beam_overlap=0.2):
        lx, ly = loop['size']

        num = loop.get('num', num)
        omit_junction = dolan.pop('omit', False)
        omit_array = array.pop('omit', False)

        junction = LJunction(**dolan)
        overlap = [
            paths.Trace(
                dolan['wire.width'],
                start=junction.node('lead0'),
                direction='-x'
            ).segment(beam_overlap),
            paths.Trace(
                dolan['wire.width'],
                start=junction.node('lead1'),
                direction='+x'
            ).segment(beam_overlap)
        ]
#         junction.add(overlap)
        junction.assign_layers([2, 3])
        
        arr = SNAILArray(**array)
        ly = arr.node('lead_1')[1]-arr.node('lead_0')[0]+27

        lp = []

        for l in range(num):
            xcenter = (l - (num - 1) / 2) * (lx + loop['width'])
            j = junction.place((xcenter,-junction.node('lead1')[1]+0.5-5.42))
            a = arr.place((xcenter-loop['size'][0]/2 - loop['width'], -(ly + array['wire.width'])/2-5.42+0.614))
            a2 = arr.place((xcenter+loop['size'][0]/2 + loop['width']+0.5, -(ly + array['wire.width'])/2-5.42+0.614))
            if not omit_junction:
                self.add(j)
            if not omit_array:
                self.add(a)
                self.add(a2)
            left = xcenter - lx / 2
            right = xcenter + lx / 2

            x, y = j.node('lead0')
            lp.append(
                Trace.between((left-1.5, y-0.92), (x-0.2, y-0.92), dolan['wire.width'])
            )

            x, y = j.node('lead1')
            lp.append(
                Trace.between((right+3.167, y+4.92), (x+0.2,y+4.92), dolan['wire.width'])
            )

            x, y = a.node('lead_0')
            x2, y2 = a2.node('lead_0')
            lp.append(
                Trace.between((x-array['wire.width']/2, y-0.5), (x2+array['wire.width']/2,y2-0.5), 1)
            )
            x, y = a.node('lead_1')
            x2, y2 = a2.node('lead_1')
            lp.append(
                Trace(1, start = (x,y), direction = '+y').segment(6.42) #left vertical line array, to big junction
            )
            lp.append(
                Trace(1, start = (x2,y2), direction = '+y').segment(6.42) 
            )

        self.add(lp)

        _, yoffset = loop.get('offset', (0, 0))

        (xmin, _), (xmax, _) = self.get_bounding_box()
        
        return {'lead0': (xmin, yoffset), 'lead1': (xmax, yoffset)}

        # width,
        # bridge,
        # taper,
        # wire,
        # bridgegap=0,
        # short=False
        

class BridgeArray(components.Component):
    __draw__ = True

    def draw(self, num, width, bridge, gap, wire, bridgegap=0, short=False):
        doses = []
        leads = []

        if not isinstance(bridgegap, Iterable):
            bridgegap = [bridgegap] * (num - 1)
        if not isinstance(bridge, Iterable):
            bridge = [bridge] * num

        junction = paths.Trace(width=wire['width'])

        junction.taper(wire['taper_length'], width)
        junction.segment(gap)

        junction.segment(bridge[0], layer=0 if short else 1)

        for bw, bg in zip(bridge[1:], bridgegap):
            junction.segment(bg)
            junction.segment(bw, layer=1)

        junction.segment(gap)
        junction.taper(wire['taper_length'], wire['width'])

        x, y = junction.current_position()

        junction.translate(-x/2, 0)

        self.add(junction)
        return {'lead_0': np.array((-x/2, 0)), 'lead_1': np.array((x/2, 0))}

def compute_Ic(omega, Ec):
    p_w = np.poly1d([5319/(2**15), 19/(2**7), 21/(2**7), 1/4, 1])
    eta = np.poly1d([1, 0])

    p_t = omega*eta - 4*Ec + Ec*eta*p_w
    eta0 = np.real(p_t.r[np.isreal(p_t.r)])[-1]

    Ej = 2*Ec/eta0**2

    Ic = qj.Ej_to_Ic(Ej)
    return Ic

if __name__ == "__main__":
    lib = gdspy.GdsLibrary()
    lib.unit=1.0e-6
    lib.precision=1e-9

    test_run = 'FD4'
    sim_evap = True

    with open(f'{test_run}/{test_run}.yaml') as f:
        wafer_params = yaml.load(f, Loader=yaml.Loader)

    draw_functions = {
        'array.yaml': make_snail_array,
        'snail.yaml': make_snail,
        'dolan.yaml': make_ldolan
    }

    for prefix, file in wafer_params['files'].items():
        print(file)
        filename = file['parameters']
        draw = draw_functions.get(filename, make_dolan)
        # print(draw)
        outfile = file['outfile']
        with open(f'{test_run}/{filename}') as f:
            params = Params(yaml.load(f, Loader=yaml.Loader))

        if 'sweep' in file:
            sweep_params = file['sweep'].keys()
            sweep_values = file['sweep'].values()

            num_params = len(sweep_params)
            
            sweeptype = file.get('sweeptype', 'product')
            sweep = product(*sweep_values) if sweeptype == 'product' else zip(*sweep_values)
            for i, vals in enumerate(sweep):
                LAYOUT = components.Component()
            
                update = {k: v for k, v in zip(sweep_params, vals)}
                print(update, draw)
                params.update(update)
                if 'RANGE' in file:
                    for k, variation in enumerate(np.linspace(*file['RANGE'])):
                        params.update({file['NAME']: variation})
                        cells = draw(
                            f'{outfile%i}_kerr_cat',
                            params,
                            simulate_evap=sim_evap
                        )
                        CURRENT = components.Component().fromcell(cells[0].flatten())
                        TEXT = gdspy.Text(('%.2f' %variation), size = 50, layer = 10)
                        LAYOUT.add(CURRENT.place((k*1000 - file['RANGE'][2]*1000/2,0)))
                        LAYOUT.add(TEXT.translate(*(k*1000 - file['RANGE'][2]*1000/2-85,-300)))
                    print(LAYOUT)
                    cells = qd.Chip()
                    cells.add_component(LAYOUT.place())
                    # cells.render('mask', include_refs = False)
                    print(cells)
                    lib.write_gds(
                        f'{test_run}/pattern_files/{outfile%i}',
                        cells=[LAYOUT.flatten()]
                    )
                else:
                    cells = draw(
                            f'{outfile%i}_kerr_cat',
                            params,
                            simulate_evap=sim_evap
                        )
                    CURRENT = components.Component().fromcell(cells[0].flatten())
                    LAYOUT.add(CURRENT.place())
                    cells = qd.Chip()
                    cells.add_component(LAYOUT.place())
                    # cells.render('mask', include_refs = False)
                    print(cells)
                    lib.write_gds(
                        f'{test_run}/pattern_files/{outfile%i}',
                        cells=[LAYOUT.flatten()]
                    )

        else:
            filename = file['parameters']
            outfile = file['outfile']
            with open(f'{test_run}/{filename}') as f:
                params = Params(yaml.load(f, Loader=yaml.Loader))

            draw = make_short if prefix == 'short' else make_dolan

            cells = draw(
                prefix,
                params,
                simulate_evap=sim_evap
            )

            lib.write_gds(
                f'{test_run}/pattern_files/{outfile}',
                cells=[cells[0].flatten()]
            )

{'sweep': {'loop.num': [1], 'loop.offset': [[0, 0]], 'array.omit': [False], 'junction.omit': [False]}, 'sweeptype': 'zip', 'parameters': 'snail.yaml', 'outfile': 'Fluxonium-check1-%s.gds'}
{'loop.num': 1, 'loop.offset': [0, 0], 'array.omit': False, 'junction.omit': False} <function make_snail at 0x137ee7ba0>
0.1
0.7
0.17
0.2
0.9849999999999999
0.1
0.7
0.17
0.2
0.9849999999999999


FileNotFoundError: [Errno 2] No such file or directory: 'FD4/pattern_files/Fluxonium-check1-0.gds'